In [73]:
import boto3
import re
import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role
from sagemaker.serializers import CSVSerializer
from sagemaker.deserializers import CSVDeserializer
import roles
import sagemaker as sage
from time import gmtime, strftime
import json

In [3]:
#Set to True if you want to train before deploy
TRAIN = False

### AWS connection startup

In [4]:
role = f"arn:aws:iam::{roles.account_ID}:role/{roles.SageMakerExecutionRole}"
sess = sage.Session()

---

### Sample data upload to S3

In [5]:
WORK_DIRECTORY = "../src/data"
prefix = "DEMO-DATA"
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

---

### TRAINING

To create the estimator we need:
* an ECR image
* a role
* an instance_type
* and output_path
* a session

In [6]:
account = sess.boto_session.client("sts").get_caller_identity()["Account"]
region = sess.boto_session.region_name
image = "{}.dkr.ecr.{}.amazonaws.com/sagemaker-deploy-terraform:latest".format(account, region)
instance_type = "ml.c4.2xlarge"
output_path = "s3://{}/output".format(sess.default_bucket())

In [7]:
# # we use the Hyperparameter Tuner
# from sagemaker.tuner import IntegerParameter

# # Define exploration boundaries
# hyperparameter_ranges = {
#     "n-estimators": IntegerParameter(20, 100),
#     "min-samples-leaf": IntegerParameter(2, 6),
# }

# # create Optimizer
# Optimizer = sagemaker.tuner.HyperparameterTuner(
#     estimator=sagemaker_model,
#     hyperparameter_ranges=hyperparameter_ranges,
#     base_tuning_job_name="RF-tuner",
#     objective_type="Minimize",
#     objective_metric_name="median-AE",
#     metric_definitions=[
#         {"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"} # Hay que hacer que el script de entrenamiento printee
#     ],  # extract tracked metric from logs with regexp
#     max_jobs=10,
#     max_parallel_jobs=2,
# )


# Optimizer.fit({"train": trainpath, "test": testpath})


# get tuner results in a df
# results = Optimizer.analytics().dataframe()
# while results.empty:
#     time.sleep(1)
#     results = Optimizer.analytics().dataframe()
# results.head()

In [8]:
if TRAIN:
    sagemaker_model = sage.estimator.Estimator(
    image,
    role,
    1,
    instance_type,
    output_path=output_path,
    sagemaker_session=sess,
    container_port=8080
)
    sagemaker_model.fit(data_location)
    # estimator.fit({"training": "s3://bucket/path/to/training/data", 
    #            "testing": "s3://bucket/path/to/testing/data"})
else:
    # Define your trained model artifact path
    model_artifact = 's3://sagemaker-us-east-2-169385451286/output/sagemaker-deploy-terraform-2023-09-06-00-09-13-615/output/model.tar.gz'
    
    # Create a SageMaker Model object
    sagemaker_model = sage.Model(
        model_data=model_artifact,
        role=role,
        image_uri=image
    )

---

### HOSTING

Deploying process may take some minutes, you can check the status in your aws sagemaker client

In [9]:
predictor = sagemaker_model.deploy(1, "ml.m4.xlarge", serializer=CSVSerializer(), deserializer=CSVDeserializer())

----!

---

### Sample trial

In [10]:
shape = pd.read_csv("../src/data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
24,setosa,4.8,3.4,1.9,0.2
37,setosa,4.9,3.6,1.4,0.1
136,virginica,6.3,3.4,5.6,2.4


In [11]:
# drop the label column in the training set
shape.drop(shape.columns[[0]], axis=1, inplace=True)
shape.sample(3)

,1,2,3,4
15,5.7,4.4,1.5,0.4
44,5.1,3.8,1.9,0.4
128,6.4,2.8,5.6,2.1


In [12]:
import itertools

a = [50 * i for i in range(3)]
b = [40 + i for i in range(10)]
indices = [i + j for i, j in itertools.product(a, b)]

test_data = shape.iloc[indices[:-1]]

In [82]:
if not TRAIN:
    sagemaker_client = boto3.client('sagemaker')
    response = sagemaker_client.list_endpoints()
    endpoints = response['Endpoints']
    for endpoint in endpoints:
        print(f"Available endpoints:\n> {endpoint['EndpointName']}")

Available endpoints:
> sagemaker-deploy-terraform-2023-09-08-14-55-34-332


In [14]:
if not TRAIN:    
    endpoint_name = 'sagemaker-deploy-terraform-2023-09-08-14-55-34-332'
    predictor = sage.predictor.Predictor(endpoint_name)

In [59]:
print(predictor.predict(
    test_data.to_csv(sep=",", header=False, index=False), 
    initial_args={'ContentType':'text/csv'}).decode("utf-8"))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



---

# Invocation with Boto3

In [ ]:
sagemaker_runtime = boto3.client('sagemaker-runtime')

In [62]:
response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    ContentType=content_type,
    Body=test_data.to_csv(sep=",", header=False, index=False)
)

In [72]:
if response["ResponseMetadata"]["HTTPStatusCode"] == 200:
    response_body = response["Body"].read().decode("utf-8")
    print(response_body)
else:
    raise Exception('An error has occured')

---

### Clean up

In [ ]:
#sess.delete_endpoint(predictor.endpoint)